In [1]:
import pandas as pd
from redfin import Redfin
from pprint import PrettyPrinter

pp = PrettyPrinter(indent=1)

client = Redfin()

# Replace 'your_file.csv' with the path to your CSV file
file_path = 'Addresses_cleaned.csv'

# Read the CSV file into a Pandas DataFrame
df = pd.read_csv(file_path)

# Display the DataFrame
df.head()


,ACCOUNT_NUM,DIVISION_CD,STREET_NUM,FULL_STREET_NAME,PROPERTY_CITY,PROPERTY_ZIPCODE,GIS_PARCEL_ID
0,38204500060410000,RES,2326,LUAU ST,MESQUITE (DALLAS CO),751503826,38204500060410000
1,38137500370290000,RES,1737,HIGHLAND ST,MESQUITE (DALLAS CO),751491534,38137500370290000
2,60259500060160000,RES,9999,AUTUMN WOOD DR,MESQUITE (DALLAS CO),75181,60259500060160000
3,60259500050250000,RES,9925,AUTUMN WOOD DR,MESQUITE (DALLAS CO),75181,60259500050250000
4,60259500060230000,RES,9999,RIPPLEWOOD DR,MESQUITE (DALLAS CO),75181,60259500060230000


In [2]:
# Remove unnesessary (DALLAS CO) from PROPERTY_CITY series
df['PROPERTY_CITY'] = df['PROPERTY_CITY'].str.replace(r'\(DALLAS CO\)', '').str.rstrip()

# Add a column with the full address
df['FullAddress'] = df['STREET_NUM'].astype(str) + ' ' + df['FULL_STREET_NAME'] + ' ' + df['PROPERTY_CITY'] + ' ' + 'TX' + ' ' + df['PROPERTY_ZIPCODE'].astype(str).str[:5]

df = df.drop_duplicates(keep=False, subset=["FullAddress"])

df.head()

C:\Users\kayle\AppData\Local\Temp\ipykernel_49248\1303363462.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['PROPERTY_CITY'] = df['PROPERTY_CITY'].str.replace(r'\(DALLAS CO\)', '').str.rstrip()


,ACCOUNT_NUM,DIVISION_CD,STREET_NUM,FULL_STREET_NAME,PROPERTY_CITY,PROPERTY_ZIPCODE,GIS_PARCEL_ID,FullAddress
0,38204500060410000,RES,2326,LUAU ST,MESQUITE,751503826,38204500060410000,2326 LUAU ST MESQUITE TX 75150
1,38137500370290000,RES,1737,HIGHLAND ST,MESQUITE,751491534,38137500370290000,1737 HIGHLAND ST MESQUITE TX 75149
3,60259500050250000,RES,9925,AUTUMN WOOD DR,MESQUITE,75181,60259500050250000,9925 AUTUMN WOOD DR MESQUITE TX 75181
5,38213500050160000,RES,2900,SIDNEY DR,MESQUITE,751502253,38213500050160000,2900 SIDNEY DR MESQUITE TX 75150
9,38139500580020000,RES,1507,RICHARD ST,MESQUITE,751491438,38139500580020000,1507 RICHARD ST MESQUITE TX 75149


In [3]:
addresses = [df['FullAddress']]

In [4]:
# initialize list for successful pulls
houses_list = []

for index, each in enumerate(addresses[0][20000:30000]):

    # reset values
    url = ''
    address = each

    try:
        # attempt to get data from each address
        response = client.search(address)
        url = response['payload']['exactMatch']['url']
        initial_info = client.initial_info(url)
        property_id = initial_info['payload']['propertyId']        

        house_data = client.above_the_fold(property_id, listing_id=None)

        baths = house_data['payload']['addressSectionInfo']['baths']
        beds = house_data['payload']['addressSectionInfo']['beds']
        sq_ft_lot = house_data['payload']['addressSectionInfo']['lotSize']
        sq_ft_interior = house_data['payload']['addressSectionInfo']['sqFt']['value']
        year_built = house_data['payload']['addressSectionInfo']['yearBuilt']
        lat = house_data['payload']['addressSectionInfo']['latLong']['latitude']
        long = house_data['payload']['addressSectionInfo']['latLong']['longitude']
        price = house_data['payload']['addressSectionInfo']['priceInfo']['amount']
        price_label = house_data['payload']['addressSectionInfo']['priceInfo']['label']
        img = initial_info['payload']['preloadImageUrls']

        # add to house_data_df
        house_data = {
            'property_id': property_id,
            'full_address': address,
            'beds': beds,
            'baths': baths,
            'year_built': year_built,
            'sq_ft_interior': sq_ft_interior,
            'sq_ft_lot': sq_ft_lot,
            'price': price,
            'price_label': price_label,
            'lat': lat,
            'long': long,
            'img_url': img
        }

        # append data to successful list here
        houses_list.append(house_data)

    except Exception as e:
        # anything that doesn't have above info gets thrown as error
        print(f'error: {e} at address: {address}')

    if (index+1) % 100 == 0:
            print(f'Processed {index+1} addresses')

error: 'baths' at address: 5198 SHOREGATE DR GARLAND TX 75043
error: 'beds' at address: 2505 HIGH STAR DR GARLAND TX 75041
error: 'baths' at address: 2422 HIGH STAR DR GARLAND TX 75041
error: 'beds' at address: 2405 HIGH HOLLOW DR GARLAND TX 75041
error: 'beds' at address: 2509 HIGH HOLLOW DR GARLAND TX 75041
error: 'beds' at address: 4718 TEALWOOD PL GARLAND TX 75043
error: 'beds' at address: 5226 COLLINGWOOD DR GARLAND TX 75043
error: 'beds' at address: 329 ELMWOOD DR GARLAND TX 75043
error: 'beds' at address: 302 ELMWOOD CT GARLAND TX 75043
error: 'beds' at address: 2709 PATRICIA LN GARLAND TX 75041
error: 'beds' at address: 4733 OAKWOOD DR GARLAND TX 75043
error: 'exactMatch' at address: 434 RIDGEGATE PL GARLAND TX 75040
error: 'beds' at address: 429 SOLITUDE DR GARLAND TX 75043
error: 'beds' at address: 4406 MAYFLOWER DR GARLAND TX 75043
error: 'beds' at address: 4614 TARRY DR GARLAND TX 75043
error: 'beds' at address: 4733 CLIFFWOOD DR GARLAND TX 75043
error: 'beds' at address: 1

In [5]:
# create df
house_data_df = pd.DataFrame(
    houses_list
)

house_data_df.head()

,property_id,full_address,beds,baths,year_built,sq_ft_interior,sq_ft_lot,price,price_label,lat,long,img_url
0,31364980,1009 SEMINARY RIDGE GARLAND TX 75043,3,2.0,1984,1953,8638,345473,Redfin Estimate,32.880847,-96.616183,[https://maps.googleapis.com/maps/api/streetvi...
1,31364996,2013 MERRIMAC TRL GARLAND TX 75043,3,1.5,1980,1841,8398,278333,Redfin Estimate,32.883196,-96.614613,[https://ssl.cdn-redfin.com/photo/90/mbphoto/1...
2,31365088,2813 HARPERS FERRY LN GARLAND TX 75043,3,2.0,1984,1974,8756,330036,Redfin Estimate,32.881430,-96.613591,[https://maps.googleapis.com/maps/api/streetvi...
3,31364927,1902 MERRIMAC TRL GARLAND TX 75043,4,2.0,1984,2289,16152,424025,Redfin Estimate,32.880954,-96.612235,[https://ssl.cdn-redfin.com/photo/90/mbphoto/3...
4,31364924,1918 MERRIMAC TRL GARLAND TX 75043,4,2.0,1988,2558,19741,384529,Redfin Estimate,32.881672,-96.612339,[https://ssl.cdn-redfin.com/photo/90/mbphoto/4...


In [6]:
# extract zip code into a new column
house_data_df['zip_code'] = house_data_df['full_address'].str.slice(-5)

house_data_df.head()

,property_id,full_address,beds,baths,year_built,sq_ft_interior,sq_ft_lot,price,price_label,lat,long,img_url,zip_code
0,31364980,1009 SEMINARY RIDGE GARLAND TX 75043,3,2.0,1984,1953,8638,345473,Redfin Estimate,32.880847,-96.616183,[https://maps.googleapis.com/maps/api/streetvi...,75043
1,31364996,2013 MERRIMAC TRL GARLAND TX 75043,3,1.5,1980,1841,8398,278333,Redfin Estimate,32.883196,-96.614613,[https://ssl.cdn-redfin.com/photo/90/mbphoto/1...,75043
2,31365088,2813 HARPERS FERRY LN GARLAND TX 75043,3,2.0,1984,1974,8756,330036,Redfin Estimate,32.881430,-96.613591,[https://maps.googleapis.com/maps/api/streetvi...,75043
3,31364927,1902 MERRIMAC TRL GARLAND TX 75043,4,2.0,1984,2289,16152,424025,Redfin Estimate,32.880954,-96.612235,[https://ssl.cdn-redfin.com/photo/90/mbphoto/3...,75043
4,31364924,1918 MERRIMAC TRL GARLAND TX 75043,4,2.0,1988,2558,19741,384529,Redfin Estimate,32.881672,-96.612339,[https://ssl.cdn-redfin.com/photo/90/mbphoto/4...,75043


In [7]:
house_data_df.value_counts(['price_label'])

price_label    
Redfin Estimate    8346
dtype: int64

In [8]:
house_data_df.dtypes

property_id         int64
full_address       object
beds                int64
baths             float64
year_built          int64
sq_ft_interior      int64
sq_ft_lot           int64
price               int64
price_label        object
lat               float64
long              float64
img_url            object
zip_code           object
dtype: object

In [9]:
house_data_df.shape

(8346, 13)

In [10]:
# export df as csv
house_data_df.to_csv('./redfin_house_data_batch3.csv', index=False)